In [1]:
import joblib

In [2]:
train_embeddings = joblib.load('train_embeddings.joblib')

In [3]:
len(train_embeddings[3]['mlp_0'][0][1])

4096

In [4]:
import umap

reducer = umap.UMAP(n_components=128)

In [5]:
import numpy as np

to_transform = []
for item in train_embeddings:
    for layer_num in [0, 5, 12, 19, 26]:
        layer_name = f'mlp_{layer_num}'
        for embedding in item[layer_name][0]:
            to_transform.append(np.array(embedding))

In [6]:
to_transform[1].shape

(4096,)

In [7]:
reducer.fit(to_transform)

UMAP(n_components=128, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [8]:
import numpy as np

umap_embeddings = []

for item in train_embeddings:
    umap_embedding = {}
    for layer_num in [0, 5, 12, 19, 26]:
        layer_name = f'mlp_{layer_num}'
        umap_embedding[layer_name] = reducer.transform(np.array(item[layer_name][0]))
    
    umap_embedding['logits'] = np.array(item['logits'])
    umap_embedding['input_ids'] = np.array(item['input_ids'])
    
    umap_embeddings.append(umap_embedding)

/home/alce/src/the_chatbot_experiment/.env/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [9]:
umap_embeddings[1]['mlp_0'].shape

(66, 128)

In [10]:
joblib.dump(umap_embeddings, 'umap_embeddings.joblib')

['umap_embeddings.joblib']

### Training from embeddings

In [1]:
import joblib

umap_embeddings = joblib.load('umap_embeddings.joblib')

In [2]:
import torch
import transformers
import torch.nn.functional as F

from tqdm import tqdm
from transformers.models.gptj.modeling_gptj import GPTJForCausalLM, GPTJConfig

In [3]:
config = transformers.GPTJConfig(n_embd=128,
                                 n_layer=6,
                                 n_head=4)

In [4]:
gpt_small = GPTJForCausalLM(config)

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [6]:
def create_text_from_summary_and_dialogue(summary, dialogue):
    text = f"""
A partial summary of the conversation is:
{summary}

With the dialogue being:
{dialogue}
    """.strip()
    
    return text.replace('\r\n', '\n')

In [7]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output
    return hook

for layer_num in range(11):
    gpt_small.transformer.h[layer_num].mlp.register_forward_hook(get_activation(f'mlp_{layer_num}'))

In [8]:
summary = """
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago.
Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are.
John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes. 
The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital.
If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".
""".strip().replace('\n', ' ')

dialogue = """
Alberto: What happened to my lasagne?
John: 
""".strip()

text = create_text_from_summary_and_dialogue(summary, dialogue)

prompt = tokenizer(text, return_tensors='pt').to('cuda')
prompt = {key: value for key, value in prompt.items()}

In [9]:
from datasets import load_dataset


#gpt_small.gradient_checkpointing_enable()

optimizer = torch.optim.Adam(gpt_small.parameters(), lr=1e-3)

epochs = 30
gpt_small.cuda()

for epoch_num in range(epochs):
    gpt_small.train()

    for item in tqdm(umap_embeddings):            
        input_ids = torch.tensor([item['input_ids']]).cuda()
        label_logits = torch.tensor([item['logits']]).cuda()
        out_logits = gpt_small.forward(input_ids).logits

        out_p = F.softmax(out_logits, dim=-1)[0]
        label_p = F.softmax(label_logits, dim=-1)[0]

        loss = -torch.mean(torch.mul(torch.log(out_p).flatten(),
                                    label_p.flatten()))
        loss -= torch.mean(torch.mul(torch.log(1 - out_p).flatten(),
                                    (1 - label_p).flatten()))

        for small_index, big_index in enumerate([0, 5, 12, 19, 26]):
            small_layer_name = f'mlp_{2 * small_index}'
            big_layer_name = f'mlp_{big_index}'
            loginput = F.log_softmax(activation[small_layer_name], dim=-1)
            logtarget = F.log_softmax(torch.tensor([item[big_layer_name]]).cuda(), dim=-1)
            loss += 0.1 * F.kl_div(
                loginput,
                logtarget,
                log_target=True,
            )

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    print("Epoch", epoch_num)
    gpt_small.eval()
    out = gpt_small.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 20, do_sample=False)
    print(tokenizer.decode(out[0]))

  0%|                                                    | 0/43 [00:00<?, ?it/s]/home/alce/src/the_chatbot_experiment/.env/lib/python3.8/site-packages/torch/nn/functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
100%|███████████████████████████████████████████| 43/43 [00:09<00:00,  4.74it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 0
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my lasagne?
John:






















100%|███████████████████████████████████████████| 43/43 [00:08<00:00,  4.80it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 1
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my lasagne?
John:






















100%|███████████████████████████████████████████| 43/43 [00:08<00:00,  4.89it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Epoch 159
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my lasagne?
John: I'm in a last episode of forks?













100%|███████████████████████████████████████████| 43/43 [00:09<00:00,  4.76it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 160
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my lasagne?
John: I'll be a some wine.















 51%|██████████████████████                     | 22/43 [00:04<00:04,  4.82it/s]


KeyboardInterrupt: 

In [28]:
_ = gpt_small.eval()

In [29]:
%%time

summary = """
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago.
Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are.
John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes. 
The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital.
If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".
""".strip().replace('\n', ' ')

dialogue = """
Alberto: What happened to my lasagne?
John: 
""".strip()

text = create_text_from_summary_and_dialogue(summary, dialogue)

prompt = tokenizer(text, return_tensors='pt').to('cuda')
prompt = {key: value for key, value in prompt.items()}
out = gpt_small.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 10, do_sample=False)
print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my lasagne?
John:lulululululululululu
CPU times: user 64.2 ms, sys: 19 µs, total: 64.2 ms
Wall time: 63.6 ms


In [ ]:
gpt_small.save_pretrained('./gptj_small')